In [3]:
import numpy as np
import matplotlib.pyplot as plt
import json

from scipy.signal import find_peaks, medfilt, correlate, correlation_lags, butter, sosfilt

#import PyQt5 # для графиков в отдельных картинках
#%matplotlib qt  

%matplotlib inline
#plt.rcParams['figure.figsize'] = [16, 1.5]
np. set_printoptions(suppress=True)


In [2]:
class signal:
    '''Базовый класс для сигналов'''  
    
    list_of_2 = [2**5, 2**6, 2**7, 2**8, 2**9, 2**10, 2**11, 2**12, 2**13, 2**14, 2**15, 2**16, 2**17, 2**18, 2**19, 2**20]
   
    def __init__(self, sig):
        self.data = sig
        self.fd = 409
        self.len =  self.data.size  
                      
        
   
    def plot_signal(self):
        '''Построение графика сигнала'''
        plt.plot(self.data)
        #plt.title('Сигнал')
        #plt.xlabel('Индекс')
        plt.ylabel('Амплитуда, мВ')
        plt.xticks(np.arange(0, self.len, self.len/30), rotation=45);
        plt.grid(True)
        plt.show()

        
   

In [3]:
class signal_channel(signal):
    '''Сигнал с одного канала'''  
    
    def split_signal(self, time_limits, offset=0):
        '''Разбивка основной записи на испытания'''
        data_split = []
        for i in range(len(time_limits)):
            data_split.append(signal_segment( self.data[ time_limits[i][1] + offset : time_limits[i][2] + offset]))
        self.data_split = data_split
  

In [4]:
class signal_segment(signal):
    '''Отрезки основной записи сигнала, полученные с помощью time_limits'''
    
    def calculate_full_spectrum(self, peaks_height = 0.5):
        '''Высчитывает спектр сигнала и находит его пики не ниже чем peaks_height'''
        len_2 = min(self.list_of_2, key=lambda x:  (len(self.data)-x) if (len(self.data)-x)>=0 else np.nan) # максимальное значение 2^n меньше или равное заданному
        
        spectrum = np.abs(np.fft.rfft(self.data, n = len_2, norm='forward',))*2
        spectrum[0] = spectrum[0]/2 # нормализация нулевой частоты
        spectrum = spectrum * 2 # перевод амплитуды в размах        
        frequency_array = np.fft.rfftfreq(len_2, d = 1./self.fd)
        self.spectrum = spectrum
        self.frequency_array = frequency_array
        
        self.peaks_index, _ = find_peaks(self.spectrum, height=peaks_height)
        #print('Найденные амплитуда и частота пика:',np.around(self.spectrum[self.peaks_index],3), np.around(self.frequency_array[self.peaks_index],3))
        #return spectrum, frequency_array
        
    
    def calculate_window_spectrum(self, n = 2**11, step = 256, peaks_height = 0.5):
        '''Высчитывает массив с максимальными компонентами спектра для скользящего окна размера n с шагом step,
        Возвращает массив - первый столбец - амплитуда, второй - частота, третий - смещение окна относительно начала записи '''
        
        self.window_size = n
        self.step = step
        begin = 0
        end = begin + n
        result_array = [] 
        data = self.data
        
        for offset in np.arange(0, len(data)-n, step):
    
            sig_window = signal_segment(data[begin + offset: end + offset])
            sig_window.calculate_full_spectrum(peaks_height = peaks_height);
            
            # Фильтрация по количеству найденных компонент (от 1 до 2 включительно), для отбрасывания переходных участков
            if (sig_window.peaks_index.size>0) and (sig_window.peaks_index.size<=2):
                result_array.append([
                    np.max(sig_window.spectrum[sig_window.peaks_index]),
                    sig_window.frequency_array[sig_window.peaks_index[(sig_window.spectrum[sig_window.peaks_index]).argmax()]],
                    offset,   
                    ])                            
            else:
                result_array.append([0,0, offset]) 
        self.window_spectrum_array = np.array(result_array)
        
        #return self.window_spectrum_array       
    
    
    def plot_spectrum_peaks(self):
        '''Построения графика спектра с пиками, если они были найдены'''
        #print('len:', self.spectrum.size)
        plt.plot(self.frequency_array, self.spectrum)
        #plt.title('Спектр')
        #plt.xlabel('Частота, Гц')
        plt.ylabel('Размах, мВ')
        plt.grid(True)
        plt.plot(self.frequency_array[self.peaks_index], self.spectrum[self.peaks_index],'x')
        for index in self.peaks_index:
            plt.text(self.frequency_array[index], self.spectrum[index], str(round(self.spectrum[index],2))+'mV, ' + str(round(self.frequency_array[index],3))+ ' Hz', size=14)
        plt.show()        
    
    
    
    def plot_window_spectrum(self):
        '''Построение графиков амплитуды и частоты главной компоненты в окне'''
        
        #window_info_str = ' (window: ' + str(round(self.window_size/409,2)) + ' sec, ' + 'step: ' +  str(round(self.step/409,2)) + ' sec)'
        if self.window_spectrum_array.size !=0:
    
            grap = self.window_spectrum_array[:,0]
            freq = self.window_spectrum_array[:,1]
            plt.plot(self.window_spectrum_array[:,2]/409 , grap)
            plt.plot(self.window_spectrum_array[:,2]/409 , grap, 'x')
            #plt.title('Измеренный размах в окне' + window_info_str, loc='left', size=12)
            #plt.xlabel('Начало окна, с', size=12)
            plt.ylabel('Размах, мВ')
            plt.grid(True)
            #plt.ylim(np.mean(grap)-2, np.mean(grap)+2)
            
            if len(grap)//5 == 0:
                step = 1
            else:
                step = len(grap)//5
            
            for index in np.arange(0, len(grap), step):
                plt.text(index*self.step/409, grap[index],  str(round(grap[index],2))+'mV, ' + str(round(freq[index],2))+'Hz' , size=12, rotation=15)  

        #plt.tight_layout()
        plt.show() 
        

In [5]:
class record:
    '''Вся многоканальная запись'''   
    def __init__(self, path, channels_number, holter_type, amplitude_convert = 1):
        
        self.number_ch = channels_number
        self.channels = []
              
        if holter_type == 'HE3':
            time_limits_file_name = r'time_limits/time_limits_HE3.json'
            base_signal_file_name = r"base_signals/sig_base_HE3.txt"
            self.base_signal = -np.loadtxt(base_signal_file_name, dtype=int, skiprows=0, usecols=0)
                
        if holter_type == 'HE3BP':
            time_limits_file_name = r'time_limits/time_limits_HE3BP.json'
            base_signal_file_name = r"base_signals/sig_base_HE3BP.txt"
            self.base_signal = -np.loadtxt(base_signal_file_name, dtype=int, skiprows=0, usecols=0)[0:196000]        
                
        with open(time_limits_file_name, 'r') as f:    
             self.time_limits = (json.load(f))
                        
        for i in range(channels_number):
            data = np.loadtxt(path, dtype=int, skiprows=0, usecols=i)
            data = -data/amplitude_convert
            self.channels.append(signal_channel(data, ))
      
    
    def find_index_lag(self, plot_corr = False):
        '''Получение смещения сигнала относительно базового сигнала. Смотрится первый канал у базовой записи и первый у текущей'''
        corr = correlate(self.channels[0].data, self.base_signal ,mode='full')
        lags = correlation_lags(len(self.channels[0].data), len(self.base_signal) , mode='full')
        #peaks_index, _ = find_peaks(corr)
        #self.index_lag = lags[peaks_index[(corr[peaks_index]).argmax()]]
        self.index_lag = lags[np.argmax(corr)]
        if plot_corr:
            plt.plot(corr)
            plt.grid(True)
        
        return self.index_lag
    
    
    def find_segment_limits(self, begin, end, border = 0.5 ):
        '''постриение графика отрезка по индесам с расширенными сигментами по бокам'''
        border = int((end-begin)*border)
        print(f'begin , end: {begin}, {end}')
        print('border:', border)

        fig, axs = plt.subplots(3, 1, figsize=(16, 8))
        for i_ch in range(3):
            temp = self.channels[i_ch].data[ begin - border: end + border]

            axs[i_ch].plot(np.arange(begin-border, end + border), temp)
            axs[i_ch].axvline(x=begin, color='red')
            axs[i_ch].axvline(x= end , color='red')
            axs[i_ch].grid(True)
            axs[i_ch].set_ylabel(f'{i_ch+1} канал, мВ')
            axs[i_ch].set_xticks(np.arange(begin-border, end+ border, len(temp)/50));
            axs[i_ch].tick_params(labelrotation=45)

        temp = []
    
    
    def plot(self, save_fig = False, save_fig_path = r"qqq1.png"):
        '''Постоение графика всей записи'''
        fig, axs = plt.subplots(3, 1, figsize=(16, 9))
        fig.subplots_adjust(wspace=0.07, hspace=0)
        for i_ch in range(3):
            temp = self.channels[i_ch].data
            axs[i_ch].plot(temp)
            axs[i_ch].grid(True)
            axs[i_ch].set_ylabel(f'{i_ch+1} канал, мВ')
            axs[i_ch].set_xlabel('index')
        temp = []

        if save_fig: 
            fig.savefig(save_fig_path)
            print('saved:', save_fig_path)
            
        print('len:',self.channels[0].len)
    

## Функции

In [6]:
def get_index_from_time(time_start, time_end, offset):
    '''Получение индекса границы отрезка по разности времени'''
    from datetime import datetime as dt 
    
    offset = (offset/1000)
    time_start_obj = dt.strptime(time_start, '%H:%M:%S')
    time_end_obj = dt.strptime(time_end, '%H:%M:%S')
    #print('index:' ,round(((time_end_obj - time_start_obj).total_seconds()+offset)*409))
    return round(((time_end_obj - time_start_obj).total_seconds()+offset)*409)

In [14]:
def nose_test(record, max_nose = 0.02, filtred = False):
    '''Испытание на шум
    считается максимальных размах шума в двадцати отрезках по 10 сек
    На вход отрезок сигнала с нужной записью
    Если включена фильтрация то начало участка обрезается '''
    
    fig, axs = plt.subplots(3, 1, figsize=(16, 6))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    i_seg = 0
    data_max_nose = 0
    rez_array = []
    for i_ch in range(3):
        temp = record.channels[i_ch].data_split[i_seg]
        data = temp.data
        
        if filtred:
            sos = butter(1, 0.1, 'hp', fs=409, output='sos')
            data = sosfilt(sos, data)
        
        axs[i_ch].plot(np.arange(temp.data.size)/409, data)
        axs[i_ch].grid(True)
        axs[i_ch].set_ylabel(f'{i_ch+1} канал, мВ')
        axs[i_ch].set_xlabel('sec')    
   
        
        
        
        if filtred: 
            # убираем 15 секуд начала записи
            begin_offset = 15*409
            offset_range = range( begin_offset, (temp.len - 10*409),  round((temp.len-begin_offset)/20))
        else:
            offset_range = range( 0, (temp.len - 10*409),  round(temp.len/20))

            
        for offset in offset_range: 
            temp_window = data[0+offset: 10*409 + offset]

            temp_window_diff = (temp_window.max() - temp_window.min())

            if temp_window_diff > data_max_nose:
                data_max_nose = temp_window_diff

            temp_window_diff = 0
            temp_window = []
        rez_array.append(round(data_max_nose,3))
        data_max_nose = 0
    temp = []
    print('Максимальный размах записи:   ', round(max(rez_array),3), 'мВ')
    print('Максимально допустимый размах:', max_nose, 'мВ')
    print(rez_array)
    return  rez_array

In [8]:
def amplitude_test(record_3ch, n_seg, peaks_height = 0.5, window_size = 2**11):
    print('Segment:' ,record_3ch.time_limits[n_seg][0])
    rez_array = []
    fig, axs = plt.subplots(3, 1, figsize=(16, 6))
    fig.subplots_adjust(wspace=0.07, hspace=0)

    for i_ch in range(3):
        temp = record_3ch.channels[i_ch].data_split[n_seg]
        axs[i_ch].plot(np.arange(temp.data.size)/409, temp.data)
        axs[i_ch].grid(True)
        axs[i_ch].set_ylabel(f'{i_ch+1} канал, мВ')
        axs[i_ch].set_xlabel('sec')
    temp = []

    for i_ch in range(3):
        record_3ch.channels[i_ch].data_split[n_seg].calculate_full_spectrum(peaks_height = peaks_height)
        record_3ch.channels[i_ch].data_split[n_seg].calculate_window_spectrum(peaks_height = peaks_height, n = window_size)
        #print(record_3ch.channels[i_ch].data_split[n_seg].window_size)

    fig, axs = plt.subplots(3, 2, figsize=(16, 6))

    base_freq = 0
    for i_ch in range(3):
        temp = record_3ch.channels[i_ch].data_split[n_seg]
        axs[i_ch,0].plot(temp.frequency_array, temp.spectrum)
        axs[i_ch,0].grid(True)
        axs[i_ch,0].set_ylabel(f'{i_ch+1} канал, мВ', rotation=90)
        axs[i_ch,0].set_xlabel('Hz')
        for index in temp.peaks_index:
            axs[i_ch,0].text(temp.frequency_array[index], temp.spectrum[index]/2, str(round(temp.spectrum[index],2))+'mV, '+ '\n' + str(round(temp.frequency_array[index],2))+ ' Hz', size=14)
            base_freq = temp.frequency_array[index]
            rez_array.append([temp.spectrum[index], temp.frequency_array[index]])

    for i_ch in range(3):
        temp = record_3ch.channels[i_ch].data_split[n_seg]    
        if temp.window_spectrum_array.size !=0:
            grap = record_3ch.channels[i_ch].data_split[n_seg].window_spectrum_array[:,0]
            freq = record_3ch.channels[i_ch].data_split[n_seg].window_spectrum_array[:,1]
            axs[i_ch,1].plot(temp.window_spectrum_array[:,2]/409 , grap)
            axs[i_ch,1].plot(temp.window_spectrum_array[:,2]/409 , grap, 'x')
            axs[i_ch,1].grid(True)
            axs[i_ch,1].set_ylabel('Размах, мВ')
            axs[i_ch,1].set_xlabel(f'sec, (размер окна: {window_size}, {round(window_size/409,2)} sec) ')
            axs[i_ch,1].set_ylim(np.mean(grap)-2, np.mean(grap)+2)
            
            if len(grap)//5 == 0:
                step = 1
            else:
                step = len(grap)//5
                
            for index in np.arange(0 ,len(grap), step):
                if abs(freq[index] - np.median(freq)) > base_freq*0.03:
                    text_str = str(round(grap[index],2))+'mV, ' + '\n' + str(round(freq[index],2))+'Hz' 
                else:
                    text_str = str(round(grap[index],2))+'mV'
                
                axs[i_ch,1].text(index*temp.step/409, grap[index],  text_str , size=12, rotation=25)      
    print(rez_array)
    return np.array(rez_array)
#fig.savefig(r"qqq1.png",)

In [1]:
def amplitude_test_v2(record, n_seg,  n_size = 5, peaks_height = 0.5, window_size = 2**11, facecolor = 'white'):
    '''Амплитудный тест, строит графики отрезка сигнала по 3 каналам с добавлением значений амплитуд и частот,
    возвращает массив с медианным значением последних n_size значений оконного спектра для каждого канала'''
    
    fig, axs = plt.subplots(3, 1, figsize=(16, 6))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    fig.suptitle('Segment: ' + record.time_limits[n_seg][0] , fontsize=14)
    fig.patch.set_facecolor(facecolor)
    #rez_array = np.empty((0, n_size), int)
    rez_array = []
    median_array = []
    
    for i_ch in range(3):
        temp = record.channels[i_ch].data_split[n_seg]

        axs[i_ch].plot(np.arange(temp.data.size)/409, temp.data)

        axs[i_ch].grid(True)
        axs[i_ch].set_ylabel(f'{i_ch+1} канал, мВ')
        #axs[i_ch].set_xlabel('sec')
        #axs[i_ch].xaxis.set_visible(False) 
        temp.calculate_window_spectrum(peaks_height = peaks_height, n = window_size)

        
        
        if temp.window_spectrum_array.size !=0:

            grap = temp.window_spectrum_array[:,0]
            freq = temp.window_spectrum_array[:,1]
            #axs[i_ch].plot(temp.window_spectrum_array[:,2]/409 , grap)
            #axs[i_ch].plot(temp.window_spectrum_array[:,2]/409 , grap, 'x')
            
            rez_array.append(np.array(grap[-n_size:]))
                    
   
            if len(grap)//6 == 0:
                step = 1
            else:
                step = len(grap)//6
            
            axs[i_ch].set_ylim(top = axs[i_ch].get_ylim()[1]*1.3)
            for index in np.arange(0 ,len(grap), step):
                text_str = str(round(grap[index],2))+'mV, ' + '\n' + str(round(freq[index],2))+'Hz' 
                limit_diff = axs[i_ch].get_ylim()[1] - axs[i_ch].get_ylim()[0]
                text_y = axs[i_ch].get_ylim()[1] - limit_diff * 0.3                
                axs[i_ch].text(index*temp.step/409, text_y,  text_str , size=12, rotation=15)  
        else: 
            for i in range(3):
                rez_array.append(0)
            
            #axs[i_ch].set_ylim(np.mean(grap)-2, np.mean(grap)+2)
    #axs[2].xaxis.set_visible(True) 
    axs[2].set_xlabel(f'sec, (размер окна: {window_size}, {round(window_size/409,2)} sec) ')
    temp = []

    #fig.tight_layout()   
    
    #fig.savefig(r"qqq1.png",)
    
    for i in range(3):
        median_array.append(round(np.median(rez_array[i]),2))
    
    print(median_array)
    return median_array

In [9]:
def ECG_test(record_3ch):
    '''Тест сегмента с ЭКГ, считает точность по времени и выводит график отрезка с переключением полярности каналов,
    возвращает результат пргохождения теста на точность времени'''
    warnung_flag = 0
    
    n_seg = 8
    fig, axs = plt.subplots(3, 1, figsize=(16, 6))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    for i_ch in range(3):
        temp = record_3ch.channels[i_ch].data_split[n_seg].data
        peaks_index, _ = find_peaks(temp, distance = 0.95*409, height = 1)
        axs[i_ch].plot(np.arange(temp.size)/409, temp)
        axs[i_ch].plot(peaks_index/409, temp[peaks_index], 'x')
        axs[i_ch].grid(True)
        axs[i_ch].set_ylabel(f'{i_ch+1} канал, мВ')
        axs[i_ch].set_xlabel('sec')

        print(f' max = {round(max(np.diff(peaks_index))/409,3)} sec, min = {round(min(np.diff(peaks_index))/409,3)} sec')
        if abs((max(np.diff(peaks_index))/409)-1.0 > 0.05)  or  abs((min(np.diff(peaks_index))/409)-1.0 > 0.05):
            warnung_flag = 1
    temp = []        

    print('Прохождение испытания:', warnung_flag==0)

    n_seg = 9
    fig, axs = plt.subplots(3, 1, figsize=(16, 6))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    for i_ch in range(3):
        temp = record_3ch.channels[i_ch].data_split[n_seg].data
        axs[i_ch].plot(np.arange(temp.size)/409, temp)
        axs[i_ch].grid(True)
        axs[i_ch].set_ylabel(f'{i_ch+1} канал, мВ')
        axs[i_ch].set_xlabel('sec')
    temp = []
    return warnung_flag == 0

In [ ]:
def doptok_test(record_3ch):
    '''Испытание на дополнительный ток в цепи пациента, считает максимальный размах сигнала на канале и сравнивает с 1 мВ'''
    n_seg = 10

    fig, axs = plt.subplots(3, 1, figsize=(16, 6))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    print('Segment:' ,record_3ch.time_limits[n_seg][0])

    ymax = 0
    ymin = 0
    warnung_flag = 0

    for i_ch in range(3):
        temp = record_3ch.channels[i_ch].data_split[n_seg].data
        temp = temp - np.median(temp)
        temp_max = max(temp)
        temp_min = min(temp)
        if ymax < temp_max * 1.5:    ymax = temp_max * 1.5   
        if ymin > temp_min * 1.5:    ymin = temp_min * 1.5

        if (abs(temp_max) > 1.0) or (abs(temp_min) > 1):
            warnung_flag = 1

        print((f'{i_ch+1} канал, max = {round(temp_max,3)} мВ, min = {round(temp_min,3)} мВ'))

    print('Прохождение испытания:', warnung_flag==0)

    for i_ch in range(3):
        temp = record_3ch.channels[i_ch].data_split[n_seg].data
        #temp = temp - np.median(temp) # Корректировка уровня (удаление постоянной составляющей)
        axs[i_ch].plot(np.arange(temp.size)/409, temp)
        axs[i_ch].grid(True)
        axs[i_ch].set_ylabel(f'{i_ch+1} канал, мВ')
        axs[i_ch].set_xlabel('sec')
        axs[i_ch].set_ylim([ymin + np.median(temp), ymax + np.median(temp)])

    temp = []
    return warnung_flag == 0

In [ ]:
def impedanse_test_a(record, n_size = 5 , n_seg = 11, imp_ch = 0, peaks_height = 0.5, window_size = 2**11, facecolor = 'white', ):
    '''Часть теста входных сопротивлений самостоятельно не применяется, но используется в функции impedanse_test(),
    строит график для одного электрода, imp_ch задает порядок электрода для смены цвета'''
    
    
    #rez_array = np.empty((0, n_size), int)
    rez_array = []
    fig, axs = plt.subplots(3, 2, figsize=(16, 6))
    fig.subplots_adjust(wspace=0.07, hspace=0)
    #fig.suptitle('Segment: ' + record.time_limits[n_seg][0] , fontsize=14)
    fig.patch.set_facecolor(facecolor)

    for offset_seg in range(2):
        n_seg = n_seg + offset_seg

        axs[0, offset_seg].set_title('Segment: ' + record.time_limits[n_seg][0] , fontsize=12)
        for i_ch in range(3):
            temp = record.channels[i_ch].data_split[n_seg]
            plot_color = None
            if imp_ch == i_ch: plot_color = 'tomato'

            axs[i_ch, offset_seg].plot(np.arange(temp.data.size)/409, temp.data, color=plot_color)
            axs[i_ch, offset_seg].grid(True)
            #axs[i_ch, offset_seg].set_ylabel(f'{i_ch+1} канал, мВ')
            #axs[i_ch, offset_seg].set_xlabel('sec')
            #axs[i_ch, offset_seg].xaxis.set_visible(False) 
            temp.calculate_window_spectrum(peaks_height = peaks_height, n = window_size)

            if temp.window_spectrum_array.size !=0:

                grap = temp.window_spectrum_array[:,0]
                freq = temp.window_spectrum_array[:,1]
                
                
            
                rez_array.append(np.array(grap[-n_size:]))
                
                
                #axs[i_ch, offset_seg].plot(temp.window_spectrum_array[:,2]/409 , grap)
                #axs[i_ch, offset_seg].plot(temp.window_spectrum_array[:,2]/409 , grap, 'x')

                if len(grap)//5 == 0:
                    step = 1
                else:
                    step = len(grap)//5

                axs[i_ch, offset_seg].set_ylim(top = axs[i_ch, offset_seg].get_ylim()[1]*1.3)
                for index in np.arange(0 ,len(grap), step):
                    text_str = str(round(grap[index],2))+'mV, ' + '\n' + str(round(freq[index],2))+'Hz' 
                    limit_diff = axs[i_ch, offset_seg].get_ylim()[1] - axs[i_ch, offset_seg].get_ylim()[0]
                    text_y = axs[i_ch, offset_seg].get_ylim()[1] - limit_diff * 0.3                
                    axs[i_ch, offset_seg].text(index*temp.step/409 , text_y,  text_str , size=10, rotation=15)        

        #axs[2, offset_seg].xaxis.set_visible(True) 
        axs[2, offset_seg].set_xlabel(f'sec, (размер окна: {window_size}, {round(window_size/409,2)} sec) ')
        temp = []

    #fig.tight_layout()   
    #fig.savefig(r"qqq1.png",)
    return rez_array

In [ ]:
def impedanse_test(record_3ch, n_size = 5, start_n_seg = 11): 
    '''Основной тест на входные сопротивления'''

    list_max = []
    list_min = []

    for i in range(3):
        impedanse_array = impedanse_test_a(record_3ch, n_size = n_size, n_seg = start_n_seg + i*2, imp_ch = i, facecolor = 'white')
        temp_list_max = []
        for j in range(len(impedanse_array)):
            temp_list_max.append(np.max(impedanse_array[j]))

        list_max.append(temp_list_max)
        list_min.append([impedanse_array[i].min(), impedanse_array[i+3].min()])  

    base_impedanse = []
    base_impedanse.append(list_max[1][3])
    base_impedanse.append(list_max[0][4])
    base_impedanse.append(list_max[1][5])

    rezult_impedanse = []

    for i in range(3):
        rezult_impedanse.append([round(0.62*list_min[i][0]/(base_impedanse[i]-list_min[i][0])),  round(0.62*list_min[i][1]/(base_impedanse[i]-list_min[i][1]))])
    print(rezult_impedanse)
    return rezult_impedanse

In [ ]:
def print_rezult(tests_rezult):
    from datetime import datetime as dt
    print(f'Шум                    : {max(tests_rezult[0][1]) < 0.020} {(tests_rezult[0][1])}')
    print()
    print(f'Амплитуда 50 мкВ       : {(tests_rezult[1][1]  == [0.05, 0.05, 0.05])} {(tests_rezult[1][1])}')
    print(f'Амплитуда 10 мВ        : {np.prod(np.abs(np.array(tests_rezult[2][1]) - 10)< 10*0.05)==1 } {(tests_rezult[2][1])}')
    print(f'Амплитуда 10 мВ +300 мВ: {np.prod(np.abs(np.array(tests_rezult[3][1]) - 10)< 10*0.05)==1 } {(tests_rezult[3][1])}')
    print(f'Амплитуда 10 мВ -300 мВ: {np.prod(np.abs(np.array(tests_rezult[4][1]) - 10)< 10*0.05)==1 } {(tests_rezult[4][1])}')
    print()
    print(f'АЧХ 50 мГц             : {np.prod(tests_rezult[5][1] > np.median(tests_rezult[6][1])*0.7)==1} {(tests_rezult[5][1])}')
    print(f'АЧХ 5 Гц               : {np.prod(np.abs(np.array(tests_rezult[6][1]) - 2)< 2*0.05)==1 } {(tests_rezult[6][1])}')
    print(f'АЧХ 100 Гц             : {np.prod(tests_rezult[7][1] > np.median(tests_rezult[6][1])*0.7)==1} {(tests_rezult[7][1])}')
    print()
    print(f'ЭКГ                    : {tests_rezult[8][1] }')
    print(f'Доп. ток               : {tests_rezult[9][1] }')
    print()
    print(f'Импеданс               : {np.prod(np.array(tests_rezult[10][1]) > 50)==1} {(tests_rezult[10][1])}')
    print(f'Импеданс +300 мВ       : {np.prod(np.array(tests_rezult[11][1]) > 50)==1} {(tests_rezult[11][1])}')
    print(f'Импеданс -300 мВ       : {np.prod(np.array(tests_rezult[12][1]) > 50)==1} {(tests_rezult[12][1])}') 
    print('\nВремя окончания:' ,dt.today().strftime("%H:%M:%S"))